In [1]:
# Phu, Andrea and Watcher
# 2018 Spring
import torch
import torch.nn as nn
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from torch import FloatTensor, LongTensor
import numpy as np
import pandas as pd
import time
import os
import pickle
import string
import torch.utils.data as data_utils
import psutil
torch.manual_seed(1)

## We first read in the data

In [44]:
processed_data_path = '/data/Dropbox/judge_embedding_data_sp18'
all_data_save_path = os.path.join(processed_data_path,"finalized_all_data","all_data_dict.pkl")
finished_embedding_folder_path = os.path.join(processed_data_path,'finished_judge_embedding')


In [3]:
all_data_dict = pickle.load(open(all_data_save_path,"rb"))

In [4]:
all_data_dict.keys()

dict_keys(['README', 'data_df', 'judge_id_to_index', 'judge_index_to_id', 'topic_glove_emb', 'citation_emb', 'case_to_citation_emb_index', 'citation_name_to_emb_index'])

In [141]:
topic_glove_emb = all_data_dict['topic_glove_emb'][0]

In [5]:
judgeId2Index = all_data_dict['judge_id_to_index'][0]
judgeIndex2Id = all_data_dict['judge_index_to_id'][0]
all_data_df = all_data_dict['data_df']
all_data_df

,caseid,year,middle_part,opinion_text,songername,judgeidentificationnumber,topic,geniss,opinion_vector,judge_embed_index,Circuit,date,affirmdummy,judge_decision
0,XB0PMNQNB5G0,1891,contentMajOp,"[0, 202, 1268, 13, 0, 4600, 3, 0, 247954, 128,...","PARDEE, DON A.",1830.0,Civil Procedure,4.0,"[-0.07445931306292275, 0.05843820821632856, -0...",0,5,1891-12-07,0.0,0
1,XFL757,1891,contentMajOp,"[15, 116, 4, 9238, 7, 22547, 3, 4092, 1219, 0,...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.07101038203845275, 0.029874279749715626, -...",1,6,1891-10-06,0.0,0
2,XB0PMRQNB5G0,1891,contentMajOp,"[215, 6, 68193, 3527, 255, 9670, 5, 12375, 3, ...","PARDEE, DON A.",1830.0,Contracts,7.0,"[-0.059691082627907024, 0.10039284279069777, -...",0,5,1891-12-07,0.0,0
3,XFL7KI,1891,contentMajOp,"[5, 0, 935, 4951, 5, 8151, 0, 4490, 3, 0, 305,...","LOCKE, JAMES WILLIAM",1410.0,Civil Procedure,4.0,"[-0.13210430623456784, 0.12352108086419746, -0...",2,5,1891-11-27,0.0,0
4,XB0OI7QNB5G0,1891,contentMajOp,"[5214, 10, 894, 2091, 4297, 21, 0, 14485, 1268...","NELSON, SUSAN RICHARD",3339.0,NaN,NaN,"[-0.09034438144397865, 0.06449958495506274, -0...",3,8,NaN,0.0,0
5,XB0PHFQNB5G0,1891,contentMajOp,"[0, 377, 202, 3, 14963, 386, 92, 3, 745, 98, 0...","PARDEE, DON A.",1830.0,Torts,7.0,"[-0.06601334912248623, 0.019215826325411318, -...",0,5,1891-11-27,0.0,0
6,XB0PJRQNB5G0,1891,contentMajOp,"[202, 1268, 21, 0, 14485, 38, 14, 60389, 187, ...","BRUCE, JOHN",292.0,Corporate Law,7.0,"[-0.11053901197255082, 0.10235485285668748, -0...",4,5,1891-12-07,1.0,1
7,XFL6PE,1891,contentMajOp,"[15, 116, 4, 18428, 0, 305, 4, 0, 1399, 202, 3...","BROWN, HENRY BILLINGS",284.0,Civil Procedure,4.0,"[-0.05526258051118215, 0.049538111821086264, -...",1,6,1891-10-06,0.0,0
8,XFL6LS,1891,contentMajOp,"[3, 133, 10550, 10, 1609, 400000, 0, 58, 84, 4...","SAGE, GEORGE READ",2089.0,Patent Law,7.0,"[-0.09545400352665524, 0.10511163229642662, -0...",5,6,1891-10-06,0.0,0
9,XB0PNFQNB5G0,1891,contentMajOp,"[0, 2536, 3, 323, 1853, 128, 3, 50, 196, 7091,...","PARDEE, DON A.",1830.0,Mortgages & Liens,7.0,"[-0.08174452291734276, 0.08172237472717521, -0...",0,5,1891-12-07,1.0,1


In [65]:
cc = zzz.loc[lambda df: (df['year'] == 1891) | (df['year'] == 1916),:]

In [66]:
xx = zzz.loc[lambda df: (df['year'] == 1891) | (df['year'] == 1916)]

## Now we have the data in memory, we first produce the naive judge embedding

In [80]:
def get_naive_judge_embedding(all_data_df,judgeIndex2Id, start_year, end_year, verbose=0):
    # init empty judge embedding
    # will only count year in [start_year,end_year)
    num_judge = len(judgeIndex2Id.keys())
    naive_judge_embedding = np.zeros((num_judge,300))
    for index in range(num_judge): # for each judge index
        if verbose==1 and (index+1)%300==0:
            print(index)
        cases_of_this_judge = all_data_df.loc[all_data_df['judge_embed_index']==index]
        # only count cases in the year range
        cases_of_this_judge = cases_of_this_judge.loc[lambda df: (df['year'] >= start_year) & (df['year'] < end_year)]
        num_cases = cases_of_this_judge.shape[0]
        for c in range(num_cases):
            # use iloc to iterate through all cases of this judge
            case_opinion_vector = cases_of_this_judge.iloc[c]['opinion_vector'] # (300,)
            naive_judge_embedding[index,:] += case_opinion_vector # add to the judge embedding vector
        # now we added all cases, we need get average
        if num_cases != 0:
            naive_judge_embedding[index,:] /= num_cases
    return naive_judge_embedding

In [40]:
naive_judge_embedding = get_naive_judge_embedding(all_data_df,judgeIndex2Id,0,3000,1)

In [41]:
naive_judge_embedding.shape

(2099, 300)

In [73]:
year_ranges= [ ]
for i in range(12):
    year_ranges.append(1891 + i*10)
year_ranges.append(2014)
year_ranges

[1891, 1901, 1911, 1921, 1931, 1941, 1951, 1961, 1971, 1981, 1991, 2001, 2014]

In [81]:
all_naive_judge_embeddings = {}
all_naive_judge_embeddings['allyears'] = naive_judge_embedding

for i in range(len(year_ranges)-1):
    start_year = year_ranges[i]
    end_year = year_ranges[i+1]
    new_naive_emb = get_naive_judge_embedding(all_data_df,judgeIndex2Id,start_year,end_year,1)
    all_naive_judge_embeddings['year'+str(start_year)] = new_naive_emb
    

299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799
299
599
899
1199
1499
1799


In [83]:
all_naive_judge_embeddings.keys()

dict_keys(['allyears', 'year1891', 'year1901', 'year1911', 'year1921', 'year1931', 'year1941', 'year1951', 'year1961', 'year1971', 'year1981', 'year1991', 'year2001'])

In [87]:
naive_judge_emb_path = os.path.join(finished_embedding_folder_path,"naive_judge_emb_dict.pkl")
pickle.dump(all_naive_judge_embeddings,open(naive_judge_emb_path,'wb'))

## Now we will go for more sophisticated embeddings we will convert the data into numpy matrix and get ready for training.

## we first split the data

In [136]:
from random import shuffle
from sklearn.utils import shuffle as skshuffle

def train_val_test_split(data_df,number_judges,train_ratio=0.8,val_ratio=0.1,verbose=0,toshuffle=True):
    starttime= time.time()
    sorted_all_data = data_df.sort_values(by='judge_embed_index')
    train_indexes = []
    val_indexes = []
    test_indexes = []
    currentiloc = 0
    for judge_index in range(number_judges):
        if verbose and judge_index%500 == 0:
            print(judge_index,time.time()-starttime)
        
        cases_of_this_judge = sorted_all_data.loc[sorted_all_data['judge_embed_index'] == judge_index]
        number_cases = cases_of_this_judge.shape[0]
        n_of_train = int(number_cases*train_ratio)
        n_of_val = int(number_cases*val_ratio)
        
        nextiloc = currentiloc+number_cases
        
        indexes = [i for i in range(currentiloc, nextiloc)]
        shuffle(indexes)
        
        train_indexes += indexes[:n_of_train]
        val_indexes += indexes[n_of_train:n_of_train+n_of_val]
        test_indexes += indexes[n_of_train+n_of_val:]
        
        currentiloc = nextiloc
    return skshuffle(data_df.loc[train_indexes]),skshuffle(data_df.loc[val_indexes]),skshuffle(data_df.loc[test_indexes])

In [137]:
data_train, data_val, data_test = train_val_test_split(all_data_df,2099,verbose=1)

0 0.3536055088043213
500 2.1068778038024902
1000 3.6196415424346924
1500 5.090168714523315
2000 6.303307771682739


## Now we get opinion vector, ruling label and topic vector to form the make data into X, y numpy matrix forms

In [166]:
data_train.shape

(235456, 14)

In [167]:
def df_to_Tensor(df,topic_glove_emb,verbose=0):
    # use this to convert a dataframe to torch tensor
    feature_dim = 300+300+2
    X = np.zeros((df.shape[0],feature_dim))
    y = np.zeros(df.shape[0])
    
    for i in range(df.shape[0]):
        if verbose and i%10000==0:
            print(i)
        
        data_entry = df.iloc[i]
        
        X[i,:300] = data_entry['opinion_vector']
        topic = data_entry['topic']
        topic = str.lower(str(topic)) 
        
        if topic not in topic_glove_emb: # deal with any unknown topic
            topic = "<UNK>"
            
        X[i,300:600] = topic_glove_emb[topic]
        decision = data_entry['judge_decision']
        X[i,600+decision] = 1 # one hot representation for judge decision
        y[i] = data_entry['judge_embed_index']
        
    return FloatTensor(X),LongTensor(y)

In [168]:
X_train, y_train = df_to_Tensor(data_train,topic_glove_emb,1)
X_val, y_val = df_to_Tensor(data_val,topic_glove_emb,1)
X_test, y_test = df_to_Tensor(data_test,topic_glove_emb,1)


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
0
10000
20000
0
10000
20000
30000
